# Train a Model to Trade on the Stock Market

![](https://ei.marketwatch.com/Multimedia/2018/01/16/Photos/ZH/MW-GB819_stocks_20180116134819_ZH.jpg?uuid=d1d56e68-faed-11e7-9db3-9c8e992d421e)

## Efficient Market Hypothesis

Future prices cannot be predicted by analyzing prices from the past. Excess returns cannot be earned in the long run by using investment strategies based on historical share prices or other historical data. Technical analysis techniques will not be able to consistently produce excess returns.

 

## Technical Analysis


In finance, technical analysis is an analysis methodology for forecasting the direction of prices through the study of past market data, primarily price and volume. The efficacy of technical  analysis is disputed by the efficient-market hypothesis which states that stock market prices are essentially unpredictable.

 

## Can a neural net learn Technical Analysis techniques to predict the market and make good trades?

Are we about to get rich?

In [161]:
#from alpha_vantage.techindicators import TechIndicators
#from alpha_vantage.timeseries import TimeSeries
import random
from indicators import *
from transform import *
import pandas_datareader as web
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import h2o
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
import cufflinks as cf
from plots import *
import numpy as np
import operator
import math

# Suppress unwatned warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

# Set some defaults for how pandas displays
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Enter These Variables

In [162]:
stockSymbol = "PINC"
market = "NASDAQ"
invest = 1000
window = 15

start_date = pd.datetime(2013,1,1)
split_validation = pd.datetime(2017,1,1)
split_test = pd.datetime(2018,1,1)

### Let's use a machine learning library called H2O

In [163]:
#h2o.cluster().shutdown()

In [164]:
h2o.init(max_mem_size="12G")
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 day 8 hours 28 mins
H2O cluster timezone:,America/Toronto
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.8
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_brett_08igme
H2O cluster total nodes:,1
H2O cluster free memory:,10.23 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


### Connect to Morningstar and Download the Historical Data  

In [165]:
df = web.DataReader(stockSymbol, 'morningstar', start=start_date)
df = df.reset_index(drop=False)
df = df.drop('Symbol', axis=1)
df.set_index('Date', inplace=True)
df = df[['Open','High', 'Low', 'Close','Volume']]


### Plot.ly is a Sweet Charting Library

In [166]:
myPlotlyKey = 'conh5EnFad0Z9Lz6mVWr'
py.sign_in(username='bretto777',api_key=myPlotlyKey)

fig = simpleStockChart(df,stockSymbol,split_validation,split_test)
py.iplot(fig, validate=False)

The draw time for this plot will be slow for clients without much RAM.


### Technical Analysis
Add technical analysis indicators used by the pros! Hopefully our model will learn how these indicators can predict tomorrow's price movements.

In [167]:
%%capture
df.reset_index(drop=False, inplace=True, col_level=0)

typical_price(df, high_col = 'High', low_col = 'Low', close_col = 'Close')
money_flow_index(df, vol_col='Volume')
rsi(df, close_col='Close')
ema(df, period=26, column='Close')
ema(df, period=12, column='Close')
macd(df, period_long=26, period_short=12, period_signal=9, column='Close')
on_balance_volume(df, trend_periods=21, close_col='Close', vol_col='Volume')
price_volume_trend(df, trend_periods=21, close_col='Close', vol_col='Volume')
bollinger_bands(df, trend_periods=20, close_col='Close')
williams_r(df, periods=14, high_col='High', low_col='Low', close_col='Close')
ultimate_oscillator(df, period_1=7,period_2=14, period_3=28, high_col='High', low_col='Low', close_col='Close')
df = df.drop(['uo_bp', 'uo_tr', 'uo_avg_1', 'uo_avg_2', 'uo_avg_3'], axis=1)

In [168]:
fig = technicalAnalysisChart(df)
py.iplot(fig, validate=False)

In [169]:
%%capture
windowTransform(df, window)

In [170]:
df['Close Tomorrow Trend'].replace(0,-1,inplace=True)    

trainingDF = df.loc[df['Date'] <= split_validation]
validationDF = df[df['Date'].isin(pd.date_range(split_validation,split_test))]
testDF = df.loc[df['Date'] > split_test]

trainingDF = trainingDF.dropna(axis=0, how='any')
validationDF = validationDF.dropna(axis=0, how='any')
testDF = testDF.fillna(value=0)

trainingDF.set_index('Date', inplace=True)
validationDF.set_index('Date', inplace=True)
testDF.set_index('Date', inplace=True)

In [171]:
train = h2o.H2OFrame(trainingDF, destination_frame="train.hex")
valid = h2o.H2OFrame(validationDF, destination_frame="valid.hex")
test = h2o.H2OFrame(testDF, destination_frame="test.hex")

train[train['Close Tomorrow Trend'] == 0, 'Close Tomorrow Trend'] = -1
train['Close Tomorrow Trend'] = train['Close Tomorrow Trend'].asfactor()
train = train.drop('Close Tomorrow', axis=1)
train = train.drop('Close Tomorrow Change', axis=1)

valid[valid['Close Tomorrow Trend'] == 0, 'Close Tomorrow Trend'] = -1
valid['Close Tomorrow Trend'] = valid['Close Tomorrow Trend'].asfactor()
valid = valid.drop('Close Tomorrow', axis=1)
valid = valid.drop('Close Tomorrow Change', axis=1)

test[test['Close Tomorrow Trend'] == 0, 'Close Tomorrow Trend'] = -1
test['Close Tomorrow Trend'] = test['Close Tomorrow Trend'].asfactor()
test = test.drop('Close Tomorrow', axis=1)
test = test.drop('Close Tomorrow Change', axis=1)

# Set predictor and response variables
y = 'Close Tomorrow Trend'
x = train.columns
x.remove(y)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [173]:
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator as H2OPCA

pca_decomp = H2OPCA(k=84, transform="NONE", pca_method="gram_s_v_d", use_all_factor_levels=True)
pca_decomp.train(x=x, training_frame=train)
trainReduced = pca_decomp.predict(train)
trainReduced = trainReduced.cbind(train['Close Tomorrow Trend'])

pca_decomp.train(x=x, training_frame=valid)
validReduced = pca_decomp.predict(valid)
validReduced = validReduced.cbind(valid['Close Tomorrow Trend'])

pca_decomp.train(x=x, training_frame=test)
testReduced = pca_decomp.predict(test)
testReduced = testReduced.cbind(test['Close Tomorrow Trend'])

pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%
pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%
pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%


### Try AutoML
See what kind of accuracy we can get from AutoML

In [174]:
from h2o.automl import H2OAutoML

# Set predictor and response variables
y = "Close Tomorrow Trend"
x = trainReduced.columns
x.remove(y)

# Run AutoML until 20 models are built
autoModel = H2OAutoML(max_models = 100)
autoModel.train(x = x, y = y,
          training_frame = trainReduced,
          validation_frame = validReduced,
          leaderboard_frame = testReduced)

leaders = autoModel.leaderboard
leaders

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


model_id,auc,logloss
GBM_grid_0_AutoML_20180430_123435_model_3,0.673864,0.646881
GLM_grid_0_AutoML_20180430_123435_model_0,0.621591,1.63155
DeepLearning_grid_0_AutoML_20180430_123435_model_1,0.618466,11.5386
GBM_grid_0_AutoML_20180430_123435_model_6,0.605682,1.08444
DeepLearning_0_AutoML_20180430_123435,0.6,11.6011
GBM_grid_0_AutoML_20180430_123435_model_4,0.591477,0.692639
DeepLearning_grid_0_AutoML_20180430_123435_model_2,0.578977,17.9075
GBM_grid_0_AutoML_20180430_123435_model_2,0.570455,0.701068
GBM_grid_0_AutoML_20180430_123435_model_9,0.540909,0.696899
GBM_grid_0_AutoML_20180430_123435_model_11,0.535227,0.698106


### Train a Deep Neural Net
Grid search for a good model

In [116]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.grid.grid_search import H2OGridSearch

# Set predictor and response variables
y = "Close Tomorrow Trend"
x = trainReduced.columns
x.remove(y)

hidden_opt = [[800],
              [1000],
              [2000],
              #[200,200],
              #[25,25,25],
              #[50,75,75,50],
              #[75,100,100,75],
              #[50,300,300,50],
              [1000,500],
              #[50,50,50]
              ]
l1_opt = [0.001]
l2_opt = [0.001]
rate_opt= [0.01]
activation_opt=["Rectifier","RectifierWithDropout","Tanh", "TanhWithDropout", "Maxout", "MaxoutWithDropout"]
epochs_opt=[800,1500,2000]

hyper_parameters = {"hidden":hidden_opt, 
                    "l1":l1_opt, 
                    "l2":l2_opt,
                    "rate":rate_opt, 
                    "activation":activation_opt,
                    "epochs":epochs_opt}

search_criteria = {"strategy":"RandomDiscrete", "max_models":5, "seed":123456}



model_grid = H2OGridSearch(H2ODeepLearningEstimator(),
                           hyper_params=hyper_parameters,
                           search_criteria=search_criteria,
                           grid_id='test9')
model_grid.train(x=x, 
                 y=y,                                   
                 fast_mode=True,                   
                 stopping_rounds=10, 
                 stopping_tolerance=0,
                 stopping_metric='AUC',
                 adaptive_rate=True,                
                 training_frame=trainReduced, 
                 validation_frame=validReduced,
                 score_interval=10)

deeplearning Grid Build progress: |███████████████████████████████████████| 100%


In [43]:
#!pwd

/home/olmsteadbrett/stocks


### Take a Look at the Best Model

In [175]:
#model_gridperf1 = model_grid.get_grid(sort_by='recall', decreasing=False)
#bestModel = model_gridperf1[0]  
#h2o.save_model(model=bestModel, path="/home/olmsteadbrett/stocks/deepNet1000-500.h2o")

bestModel = h2o.get_model(leaders[0,0])


In [84]:
#bestModel = h2o.load_model(path="/home/brett/ticker/h2ogbm.h2o")

In [176]:
importances = bestModel.varimp(use_pandas=True)
importances = importances.loc[:,['variable','relative_importance']].groupby('variable').mean()
importances.sort_values(by="relative_importance", ascending=False).iplot(kind='bar', colors='#5AC4F2', theme='white')

In [177]:
cm = bestModel.confusion_matrix(valid=True)
cm = cm.table.as_data_frame()
confusionMatrix = FF.create_table(cm)
confusionMatrix.layout.height=300
confusionMatrix.layout.width=800
confusionMatrix.layout.font.size=17
py.iplot(confusionMatrix)

In [178]:
pBestModel = pd.concat([bestModel.predict(test_data=testReduced).as_data_frame(use_pandas=True).reset_index(drop=True), testDF.reset_index(drop=False)], axis =1)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [195]:
prices = df.loc[df['Date'] > split_test]
prices = prices.loc[:,['Date','Close','Close Tomorrow','Close Tomorrow Trend']]
profit = pBestModel.loc[:,['predict']]
profit = pd.concat([prices.reset_index(drop=True), profit.reset_index(drop=True)], axis =1)
profit['predict'] = profit['predict'].astype(np.float64)

profit['profit'] = invest

profit['Best Possible'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['Close Tomorrow Trend'] > 0 else 0, axis=1)

profit['Randomly Invest'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if random.randint(1,1000) > random.randint(1,1000) else 0, axis=1)
profit['profit'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['predict'] > 0 else 0, axis=1)


print "Times " + stockSymbol + " actually went up    : " + str(profit[profit['Close Tomorrow Trend']>0].count()['Close Tomorrow Trend'])
print "Times " + stockSymbol + " predicted to go up  : " + str(profit[profit['predict']>0].count()['predict'])
print "Profit                        : $" + str(profit.profit.sum())
print "Random investment             : $" + str(profit['Randomly Invest'].sum())
print "Best possible outcome         : $" + str(profit['Best Possible'].sum())

profit



Times PINC actually went up    : 44
Times PINC predicted to go up  : 73
Profit                        : $188.79745237654834
Random investment             : $12.659900753001807
Best possible outcome         : $564.4100842965618


,Date,Close,Close Tomorrow,Close Tomorrow Trend,predict,profit,Best Possible,Randomly Invest
0,2018-01-02,29.69,29.43,-1.0,1.0,-8.757157,0.000000,-8.757157
1,2018-01-03,29.43,29.18,-1.0,1.0,-8.494733,0.000000,-8.494733
2,2018-01-04,29.18,29.88,1.0,1.0,23.989034,23.989034,23.989034
3,2018-01-05,29.88,29.86,-1.0,1.0,-0.669344,0.000000,0.000000
4,2018-01-08,29.86,30.09,1.0,1.0,7.702612,7.702612,7.702612
5,2018-01-09,30.09,30.60,1.0,1.0,16.949153,16.949153,0.000000
6,2018-01-10,30.60,31.26,1.0,1.0,21.568627,21.568627,0.000000
7,2018-01-11,31.26,31.82,1.0,1.0,17.914267,17.914267,0.000000
8,2018-01-12,31.82,31.82,-1.0,1.0,0.000000,0.000000,0.000000
9,2018-01-15,31.82,32.27,1.0,1.0,14.142049,14.142049,0.000000


In [89]:
bestModel


Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_0_AutoML_20180430_110905_model_0


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0853370326465
RMSE: 0.292125029134
LogLoss: 0.33598796535
Mean Per-Class Error: 0.00455580865604
AUC: 0.999903130357
Gini: 0.999806260714
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.443647314334: 


,-1,1,Error,Rate
-1,435.0,4.0,0.0091,(4.0/439.0)
1,0.0,388.0,0.0,(0.0/388.0)
Total,435.0,392.0,0.0048,(4.0/827.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4436473,0.9948718,198.0
max f2,0.4436473,0.9979424,198.0
max f0point5,0.5113486,0.9968685,189.0
max accuracy,0.4436473,0.9951632,198.0
max precision,0.8766623,1.0,0.0
max recall,0.4436473,1.0,198.0
max specificity,0.8766623,1.0,0.0
max absolute_mcc,0.4436473,0.9903420,198.0
max min_per_class_accuracy,0.4716652,0.9931663,195.0
max mean_per_class_accuracy,0.4436473,0.9954442,198.0


Gains/Lift Table: Avg response rate: 46.92 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-04-30 11:09:11,1.282 sec,0.0,0.4990483,0.6912445,0.5,1.0,0.5308343,0.4998875,0.6929231,0.5,1.0,0.5172414
,2018-04-30 11:09:11,1.318 sec,5.0,0.4513481,0.5991179,0.9156207,2.1314433,0.1717050,0.5069657,0.7074836,0.4666079,0.6904762,0.5172414
,2018-04-30 11:09:11,1.362 sec,10.0,0.4090580,0.5229234,0.9766720,2.1314433,0.0822249,0.5076746,0.7090021,0.4905938,0.6904762,0.5057471
,2018-04-30 11:09:11,1.397 sec,15.0,0.3721500,0.4601499,0.9938003,2.1314433,0.0374849,0.5092661,0.7134501,0.5125808,0.6904762,0.5172414
,2018-04-30 11:09:11,1.432 sec,20.0,0.3455293,0.4173366,0.9974696,2.1314433,0.0181378,0.5082244,0.7119236,0.5331570,0.6904762,0.5134100
,2018-04-30 11:09:11,1.470 sec,25.0,0.3153556,0.3705592,0.9996008,2.1314433,0.0072551,0.5120086,0.7207448,0.5246914,0.0,0.5172414
,2018-04-30 11:09:12,1.526 sec,30.0,0.2921250,0.3359880,0.9999031,2.1314433,0.0048368,0.5107349,0.7190532,0.5353909,0.0,0.5095785


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
PC47,31.8576946,1.0,0.0431430
PC22,30.1567574,0.9466083,0.0408396
PC28,28.3420124,0.8896442,0.0383819
PC46,26.1743011,0.8216006,0.0354463
PC34,23.0270176,0.7228087,0.0311842
---,---,---,---
PC1,7.6698370,0.2407530,0.0103868
PC37,7.1773133,0.2252929,0.0097198
PC35,6.4381680,0.2020915,0.0087188
PC16,6.1077557,0.1917200,0.0082714



See the whole table with table.as_data_frame()
